# 링크 수집

In [2]:
with open('C:/Users/dojun\Documents/api_keys/naver_api.txt') as f:
    client_id = f.readline().replace('\n', '')
    client_secret = f.readline()

In [27]:
import requests
import pandas as pd
import urllib.request
from urllib.request import quote
from urllib.parse import urlparse
from bs4 import BeautifulSoup as bs

keyword = '서울 전통시장'
url = 'https://openapi.naver.com/v1/search/news.json?query=' + keyword + '&display=100&start=901'
html = requests.get(url,headers = {"X-Naver-Client-Id" : client_id,
                                   "X-Naver-Client-Secret" : client_secret})

json1 = html.json()
len(json1['items'])

100

In [28]:
df1 = pd.DataFrame(json1['items'])
df = pd.concat([df, df1])
len(df)

1000

In [29]:
df.head()

,title,originallink,link,description,pubDate
0,"중랑구, 설 연휴 <b>전통시장</b> 찾는 주민들에 다양한 혜택 제공",http://www.munhwa.com/news/view.html?no=202102...,https://news.naver.com/main/read.nhn?mode=LSD&...,<b>서울</b> 중랑구는 설 연휴(이달 11~14일)를 맞아 오는 10일까지 <b...,"Wed, 03 Feb 2021 10:10:00 +0900"
1,"쿠팡-공정위, 중소납품업체에 3270억원 규모 지원 나서",http://www.smartfn.co.kr/view.php?ud=202102030...,http://www.smartfn.co.kr/view.php?ud=202102030...,"또 배송 규모가 작은 영세 판매자를 지원하기 위해 ‘파트너 캐리어 프로그램’, 쿠팡...","Wed, 03 Feb 2021 09:46:00 +0900"
2,집 콕 소비는 늘어나고 수입산 줄어 쇠고기값 '껑충',http://www.mbn.co.kr/pages/news/newsView.php?n...,https://news.naver.com/main/read.nhn?mode=LSD&...,【 기자 】 한우 판매점이 밀집한 <b>서울</b> 마장동의 <b>전통시장</b>입...,"Wed, 03 Feb 2021 09:38:00 +0900"
3,쿠팡의 '상생' 프로젝트... 영세 소상공인 위해선 아낌없이 푼다,https://www.techm.kr/news/articleView.html?idx...,https://www.techm.kr/news/articleView.html?idx...,<b>전통 시장</b> 살리기 프로젝트부터 <b>서울</b>시와 협업을 통한 <b>...,"Wed, 03 Feb 2021 09:31:00 +0900"
4,"쿠팡, 공정위와 손잡고 영세 소상공인 살린다",https://www.news2day.co.kr/article/20210203500020,https://www.news2day.co.kr/article/20210203500020,"또 배송 규모가 작은 영세 판매자를 지원하기 위해 ‘파트너 캐리어 프로그램’, 쿠팡...","Wed, 03 Feb 2021 09:29:00 +0900"


In [41]:
df.to_excel('./py_data/crawling/전통시장/서울전통시장_뉴스_정보.xlsx', engine='openpyxl')

In [30]:
link2021 = []
link2020 = []
link2019 = []
link2018 = []

for i, r in df.iterrows():
    if '2021' in r[4]:
        link2021.append(r[2])
    elif '2020' in r[4]:
        link2020.append(r[2])
    elif '2019' in r[4]:
        link2019.append(r[2])
    elif '2018' in r[4]:
        link2018.append(r[2])

In [31]:
print(len(link2021))
print(len(link2020))
print(len(link2019))
print(len(link2018))

1000
0
0
0


In [33]:
ls_links = list(set(link2021))

In [34]:
len(ls_links)

1000

In [35]:
links = []
for link in ls_links:
    if 'news.naver.com' in link:
        links.append(link)

In [37]:
len(links)

474

# 텍스트 수집

In [42]:
links[:10]

['https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=105&oid=014&aid=0004577331',
 'https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=100&oid=421&aid=0005124205',
 'https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=100&oid=014&aid=0004570756',
 'https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=100&oid=025&aid=0003072070',
 'https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=100&oid=003&aid=0010309874',
 'https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=101&oid=437&aid=0000257926',
 'https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=100&oid=421&aid=0005143745',
 'https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=101&oid=421&aid=0005136371',
 'https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=102&oid=032&aid=0003055920',
 'https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=101&oid=277&aid=0004834116']

In [53]:
import requests
from bs4 import BeautifulSoup as bs
from selenium import webdriver
import time
from tqdm import tqdm
import re

regex = re.compile('.*postViewArea.*')

driver = webdriver.Chrome('./py_data/crawling/chromedriver.exe')

news = ''
for link in tqdm(links, 'crawling...'):
    
    driver.get(link)
    driver.implicitly_wait(5)
    time.sleep(1)

    try:
        html = driver.page_source
        soup = bs(html, 'html.parser')
        title = soup.find('h3', class_='tts_head').text
        text = soup.find('div', class_='_article_body_contents').text
        text1 = title + '\n' + text + '\n'
        text1 = re.sub('[^ㄱ-힣0-9., ]', '', text1)
        news += text1
    except:
        continue

print('------- done --------')

crawling...: 100%|███████████████████████████████████████████████████████████████████| 474/474 [11:54<00:00,  1.51s/it]

------- done --------


In [57]:
news[:1000]

'전통시장전봇대 주변 거미줄 전선 사라진다  오류를 우회하기 위한 함수 추가  과기부, 5년간 2조8500억 투입공중케이블 지상정비지중화 추진 공중선케이블 지상 정비 전왼쪽과 후 사진. 과학기술정보통신부는 공중케이블 지상 정비 사업에 향후 5년간 2조8500억원 규모를 투자한다. 과기정통부 제공 과학기술정보통신부는 제30차 공중케이블 정비협의회를 개최해 관계부처 합동으로 마련한 제2차 공중케이블 정비 중장기 종합계획을 확정했다고 2일 밝혔다.   2차 공중케이블 정비 종합계획에서는 국가 및 지방자치단체와 한전방송통신사업자가 향후 5년간 공중케이블 지상 정비 사업에 1조4000억원, 땅속 지중화 사업에 1조4500억원 등 총 2조8500억원 규모로 투자할 계획이다. 특히 시민안전 위험지역과 전통시장 및 주택상가 지역에 대해서는 우선적으로 투자해 정비할 예정이다.   과기정통부는 과거 단순 인구비례 물량 배분방식에서 주택가구수와 노후 주택수 기준으로 조정한 후 당초 인구 50만 이상 21개 지자체에서 27개 지자체로 확대 개편 하는 등 정비사업의 실효성을 확보한다. 지자체의 기반인프라 정비지원, 지중화 확대계획, 홍보방안 등 정비계획과 이행실적을 평가한 후 지자체별 상중하로 나눠 하로 평가된 지역의 정비금액 30를 상으로 평가된 지역으로 정비물량을 재배분 한다.   과기정통부는 그린뉴딜과 연계한 예산 1조원 매칭투자로 신기술 적용을 통해 지중화 사업을 가속화 하는 등 기반시설을 고도화 한다. 아울러 적극행정으로 정부지자체한전방송통신사업자와 협업해 학생 안전을 위한 스쿨존 지중화를 우선 추진할 계획이다.   과기정통부는 지하매설 미니트렌칭공법 적용으로 지중화 공사비용을 절감하고 현재 서울 부산 2개 시범지역 외에 대전대구광주 3대광역시에 시범사업을 추가 실시해 신공법에 대한 안전성 검증 작업을 할 계획이다. 유무선 연구개발 성과도 적용해 건물과 건물사이 건물과 전주사이 중계구간에 유선케이블을 제거하고 무선 송수신 방식으로 데이터를 주고받는 네트워크 기반고도화 시범사

In [56]:
with open('./py_data/crawling/전통시장/서울전통시장_뉴스_텍스트.txt', 'w', encoding='utf-8') as f:
    f.write(news)